In [1]:
import pandas as pd
import sys
import os.path
import numpy as np
import json
import matplotlib.pyplot as plt
from spykshrk.realtime.datatypes import Datatypes

# import bokeh.plotting
# from bokeh.plotting import figure, output_notebook, show, gridplot, reset_output
# output_notebook()

#pd.set_option('float_format', '{:,.1f}'.format)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 120)



In [2]:
#config_file = '/opt/data36/daliu/realtime/spykshrk/timing/bond.config.json'
config_file = '/opt/data36/daliu/realtime/spykshrk/test/test_animal.config.json'
config = json.load(open(config_file, 'r'))

hdf_file = os.path.join(config['files']['output_dir'],
                        '{}.rec_merged.h5'.format(config['files']['prefix']))

store = pd.HDFStore(hdf_file, mode='r')
timing = store['rec_100']

In [3]:
timing['label'].unique()

In [4]:
%%time 
# Select timing data only used for spike decoding

spk_timing = timing.query("(label == 'sim_send' | label == 'spk_dec' | label == 'spk_enc' | label == 'dec_recv') & datatype == {}".format(Datatypes.SPIKES))

In [5]:
spk_timing_pivot = spk_timing.pivot_table(values='wtime', index=['timestamp', 'ntrode_id', 'datatype'], columns='label')

# Drop timing info for spikes ignored by encoder and decoder nodes, likely because they did not meet
# the encode or decode criterion.
spk_timing_pivot = spk_timing_pivot.dropna(axis=0, how='any')


In [6]:
spk_timing_pivot

In [7]:
%%time
spk_timing_pivot.eval("sim_to_dec = spk_dec - sim_send", inplace=True)
spk_timing_pivot.eval("dec_to_enc = spk_enc - spk_dec", inplace=True)
spk_timing_pivot.eval("dec_to_decoder = dec_recv - spk_dec", inplace=True)
spk_timing_pivot.eval("sim_to_decoder = dec_recv - sim_send", inplace=True)

In [8]:
spk_timing_pivot.query('dec_to_decoder > .1')

In [9]:
sim_decoder_mean = spk_timing_pivot.loc[:, 'sim_to_decoder'].mean()
sim_decoder_med = spk_timing_pivot.loc[:, 'sim_to_decoder'].median()
sim_decoder_max = spk_timing_pivot.loc[:, 'sim_to_decoder'].max()
sim_decoder_min = spk_timing_pivot.loc[:, 'sim_to_decoder'].min()

sim_dec_mean = spk_timing_pivot.loc[:, 'sim_to_dec'].mean()
sim_dec_med = spk_timing_pivot.loc[:, 'sim_to_dec'].median()
sim_dec_max = spk_timing_pivot.loc[:, 'sim_to_dec'].max()
sim_dec_min = spk_timing_pivot.loc[:, 'sim_to_dec'].min()

dec_decoder_mean = spk_timing_pivot.loc[:, 'dec_to_decoder'].mean()
dec_decoder_med = spk_timing_pivot.loc[:, 'dec_to_decoder'].median()
dec_decoder_max = spk_timing_pivot.loc[:, 'dec_to_decoder'].max()
dec_decoder_min = spk_timing_pivot.loc[:, 'dec_to_decoder'].min()

print('sim_to_decoder mean (median) (min, max): {:.04} ({:.04}) ({:.04}, {:.04}) ms'.format(sim_decoder_mean*1000, sim_decoder_med*1000,
                                                                                            sim_decoder_min*1000, sim_decoder_max*1000))
print('sim_to_dec mean (median) (min, max): {:.04} ({:.04}) ({:.04}, {:.04}) ms'.format(sim_dec_mean*1000, sim_dec_med*1000, 
                                                                                        sim_dec_min*1000, sim_dec_max*1000))
print('dec_to_decoder mean (median) (min, max): {:.04} ({:.04}) ({:.04}, {:.04}) ms'.format(dec_decoder_mean*1000, dec_decoder_med*1000,
                                                                                           dec_decoder_min*1000, dec_decoder_max*1000))

sim_to_decoder mean (median) (min, max): 0.46 (0.444) (0.296, 18.6) ms

sim_to_dec mean (median) (min, max): 0.412 (0.399) (0.247, 16.2) ms

dec_to_decoder mean (median) (min, max): 0.0473 (0.0436) (0.00691, 18.2) ms

In [10]:
temp = spk_timing_pivot['dec_to_decoder']
np.sum(np.diff(temp.values[0:10000]))

In [11]:
plt.figure(figsize=[40,10])
plt.plot(spk_timing_pivot['dec_to_decoder'].values.transpose())
plt.ylim([-0.0005, 0.0005])
plt.xlim([0, 1000])
plt.show()

In [12]:
plt.figure(figsize=[20,10])
plt.hist(spk_timing_pivot['sim_to_decoder']*1000, 1000)

plt.hist(spk_timing_pivot['sim_to_dec']*1000, 1000)

plt.hist(spk_timing_pivot['dec_to_decoder']*1000, 1000)

plt.xlim([0,2])
plt.show()


In [13]:
plt.hist(spk_timing_pivot['sim_to_dec']*1000, 10000)

plt.show()
